### LangChain Vector Stores and Retrievers abstractions
##### These abstractions are designed to support retrieval of data from vector databases and other sources for integrations with LLM workflows.They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of Retrieval Augmented Generation (RAG)
* Documents 
* Vector Stores 
* Retrievers

##### Installations needed:
```!pip install langchain```
```!pip install langchain-chroma```
```!pip install langchain_chroma```
```!pip install langchain_groq```
```!pip install langchain_huggingface```

### Documents 
##### LangChain implements a Document abstraction which is intended to represent a unit of text and associated metadata. It has 2 attributes:
* page_content: a string representing the content.
* metadata:     a dict containing arbitrary metadata. The metadata attribute can capture information about the source of the documents, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of large document as well.

In [2]:
# Generate a sample documents with Document class

from langchain_core.documents import Document

documents=[
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness",
        metadata={"source":"mammal-pets-doc"},
    ),

    Document(
        page_content="Cats are independent pets that often enjoy their own space",
        metadata={"source":"mammal-pets-doc"},
    ),


]

In [3]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')]

In [21]:
import os 
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm=ChatGroq(groq_api_key=groq_api_key,model="groq/compound")
llm

ChatGroq(profile={}, client=<groq.resources.chat.completions.Completions object at 0x000002279FB9F530>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002279FB9F0B0>, model_name='groq/compound', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [22]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") # HuggingFace model for embedding and use subsequently

In [23]:
# Create VectorStores 
# First, all the documents converted into vectors by using this embedding technique(HuggingFace embedding) then store into Chroma DB

from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(documents,embedding=embeddings)
vectorstore




In [24]:
vectorstore.similarity_search("cat")

[Document(id='fbffcb26-a985-4590-bf19-5102e96e48e2', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(id='3b11434b-504d-480a-ab37-39900de7779d', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(id='8d7a603a-bf87-4a25-8940-fd3463670f3f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(id='ed861090-32aa-4a0e-a6a6-1c30544ccd88', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]

In [25]:
# Async query functionality supported by VectorStore
await vectorstore.asimilarity_search("cat")


[Document(id='fbffcb26-a985-4590-bf19-5102e96e48e2', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(id='3b11434b-504d-480a-ab37-39900de7779d', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(id='8d7a603a-bf87-4a25-8940-fd3463670f3f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(id='ed861090-32aa-4a0e-a6a6-1c30544ccd88', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]

In [26]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='fbffcb26-a985-4590-bf19-5102e96e48e2', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
  0.9436445236206055),
 (Document(id='3b11434b-504d-480a-ab37-39900de7779d', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
  0.9436445236206055),
 (Document(id='8d7a603a-bf87-4a25-8940-fd3463670f3f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
  1.5413694381713867),
 (Document(id='ed861090-32aa-4a0e-a6a6-1c30544ccd88', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
  1.5413694381713867)]

### Retrievers 
##### LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.
* Runnable meaning: In LangChain, a Runnable is anything that can be executed in a standard way—you: give it input, it runs, and it gives you output.
* A Runnable is a callable LangChain component that supports .invoke(), .batch(), and .stream().

##### LangChain Retrievers are Runnable, so they implement a standard set of methods (eg.synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

##### We can create a simple version of this without subclass "Retriever". If we choose what method we wish to use to retrive documents, we can create a runnable easily. We will build one around the similarity_search method:

##### Create retrievers in 2 ways:
```1.Using RunnableLambda```

```2.Vectorstores implement as_retriever method``` : Vectorstores implement an as_retriever method that will generate a Retriever, specifically as VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them.

In [27]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda #Lambda function to run this Runnable

# 1. using RunnableLambda
retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])



[[Document(id='3b11434b-504d-480a-ab37-39900de7779d', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(id='ed861090-32aa-4a0e-a6a6-1c30544ccd88', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]]

In [28]:
# the same replicate with "as_retriever" method
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

retriever.batch(["cat","dog"])

[[Document(id='3b11434b-504d-480a-ab37-39900de7779d', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(id='ed861090-32aa-4a0e-a6a6-1c30544ccd88', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]]

##### How to integrate this retriever along with chain ?

In [ ]:
#RAG 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 

message="""
Answer this question using the provided context only.

{question}

Context:
{context}

"""

prompt=ChatPromptTemplate.from_messages([("human",message)])

#here, context will get thru retriever, 
rag_chain={"context":retriever,"question":RunnablePassthrough()} | prompt | llm

response=rag_chain.invoke("tell me about dogs")
print(response.content)


Dogs are great companions, known for their loyalty and friendliness.
